In [19]:
import os

import numpy as np
from PIL import Image

import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import grad
from torchvision import transforms
from torchvision import datasets
import torchvision.datasets.utils as dataset_utils


load data

In [20]:
import torch
import torch.nn.functional as F
train1 = torch.load(r'D:\Projects\Brain\ColoredMNIST\train1.pt',weights_only=False)
train2 = torch.load(r'D:\Projects\Brain\ColoredMNIST\train2.pt',weights_only=False)
test = torch.load(r'D:\Projects\Brain\ColoredMNIST\test.pt',weights_only=False)

In [22]:
def dataset_load(raw_dataset):
    x=[]
    y=[]
    for one_data in raw_dataset:
        y.append(one_data[1])
        image=one_data[0]
        data=np.array(image)
        data=data.transpose(2,0,1)
        x.append(data)
    x=torch.Tensor(np.array(x))
    y=torch.Tensor(np.array(y))
    ds=torch.utils.data.TensorDataset(x,y)
    return ds
torch_dataset_train1=dataset_load(train1)
torch_dataset_train2=dataset_load(train2)
torch_dataset_test=dataset_load(test)

In [23]:
#load data
def make_env(raw_dataset):
    x=[]
    y=[]
    for one_data in raw_dataset:
        y.append(one_data[1])
        image=one_data[0]
        data=np.array(image)
        data=data.transpose(2,0,1)
        x.append(data)
    x=torch.Tensor(np.array(x))
    y=torch.Tensor(np.array(y))
    return {
      'images': x,
      'labels': y
    }

envs=[make_env(train1),make_env(train2),make_env(test)]

# IRM

In [24]:
#load data
def make_env(raw_dataset):
    x=[]
    y=[]
    for one_data in raw_dataset:
        y.append(one_data[1])
        image=one_data[0]
        data=np.array(image)
        data=data.transpose(2,0,1)
        x.append(data)
    x=torch.Tensor(np.array(x))
    y=torch.Tensor(np.array(y))
    return {
      'images': x,
      'labels': y
    }

envs=[make_env(train1),make_env(train2),make_env(test)]

In [29]:
# Define and instantiate the model
import argparse
import numpy as np
import torch
from torchvision import datasets
from torch import nn, optim, autograd
from spikingjelly.activation_based import neuron, encoding, functional, surrogate, layer 

parser = argparse.ArgumentParser(description='Colored MNIST')
parser.add_argument('--hidden_dim', type=int, default=800)
parser.add_argument('--l2_regularizer_weight', type=float,default=0.00110794568)
parser.add_argument('--lr', type=float, default=0.0004898536566546834)
parser.add_argument('--n_restarts', type=int, default=1)
parser.add_argument('--penalty_anneal_iters', type=int, default=100)
parser.add_argument('--penalty_weight', type=float, default=91257.18613115903)
parser.add_argument('--steps', type=int, default=80)
parser.add_argument('--grayscale_model', action='store_true')
parser.add_argument('-tau', default=2.0, type=float, help='parameter tau of LIF neuron')
parser.add_argument('-T', default=1, type=int, help='simulating time-steps')
flags = parser.parse_known_args()[0]

print('Flags:')
for k,v in sorted(vars(flags).items()):
  print("\t{}: {}".format(k, v))

final_train_accs = []
final_test_accs = []
# mnist = datasets.MNIST('~/datasets/mnist', train=True, download=True)
# mnist_train = (mnist.data[:50000], mnist.targets[:50000])
# mnist_val = (mnist.data[50000:], mnist.targets[50000:])
for restart in range(flags.n_restarts):
    print("Restart", restart)
    class MLP(nn.Module):
        def __init__(self, tau):
            super(MLP, self).__init__()
            if flags.grayscale_model:
                lin1 = nn.Linear(28 * 28, flags.hidden_dim)
            else:
                lin1 = nn.Linear(3 * 28 * 28, flags.hidden_dim)
            lin2 = nn.Linear(flags.hidden_dim, flags.hidden_dim)
            lin3 = nn.Linear(flags.hidden_dim, 1)
            for lin in [lin1, lin2, lin3]:
                nn.init.xavier_uniform_(lin.weight)
                nn.init.zeros_(lin.bias)
            self._main = nn.Sequential(lin1, neuron.LIFNode(tau=tau, surrogate_function=surrogate.ATan()), neuron.LIFNode(tau=tau, surrogate_function=surrogate.ATan()), lin3)
        def forward(self, input):
            if flags.grayscale_model:
                out = input.view(input.shape[0],-1, 3, 28 * 28).sum(dim=1)
            else:
                out = input.view(input.shape[0], -1,3 * 28 * 28)
            out = self._main(out)
            return out


    mlp=MLP(tau = flags.tau)

    # Define loss function helpers

    def mean_nll(logits, y):
        return nn.functional.binary_cross_entropy_with_logits(logits, y)

    def mean_accuracy(logits, y):
        preds = (logits > 0.).float()
        return ((preds - y).abs() < 1e-2).float().mean()

    def penalty(logits, y):
        scale = torch.tensor(1.).requires_grad_()
        loss = mean_nll(logits * scale, y)
        grad = autograd.grad(loss, [scale], create_graph=True)[0]
        return torch.sum(grad**2)

    # Train loop
    def pretty_print(*values):
        col_width = 13
        def format_val(v):
            if not isinstance(v, str):
                v = np.array2string(v, precision=5, floatmode='fixed')
            return v.ljust(col_width)
        str_values = [format_val(v) for v in values]
        print("   ".join(str_values))

    optimizer = optim.Adam(mlp.parameters(), lr=flags.lr)
    encoder = encoding.PoissonEncoder()

    pretty_print('step', 'train nll', 'train acc', 'train penalty', 'test acc')

    for step in range(flags.steps):
        for env in envs:
            logits = 0.
            for t in range(flags.T):
                encode_img = encoder(env['images'])
                res = mlp(encode_img)
                logits = logits + res
            logits = logits / flags.T
            logits=logits.squeeze()
            env['nll'] = mean_nll(logits, env['labels'])
            env['acc'] = mean_accuracy(logits, env['labels'])
            env['penalty'] = penalty(logits, env['labels'])

        train_nll = torch.stack([envs[0]['nll'], envs[1]['nll']]).mean()
        train_acc = torch.stack([envs[0]['acc'], envs[1]['acc']]).mean()
        train_penalty = torch.stack([envs[0]['penalty'], envs[1]['penalty']]).mean()

        weight_norm = torch.tensor(0.)
        for w in mlp.parameters():
            weight_norm += w.norm().pow(2)

        loss = train_nll.clone()
        loss += flags.l2_regularizer_weight * weight_norm
        penalty_weight = (flags.penalty_weight 
            if step >= flags.penalty_anneal_iters else 1.0)
        loss += penalty_weight * train_penalty
        if penalty_weight > 1.0:
            # Rescale the entire loss to keep gradients in a reasonable range
            loss /= penalty_weight

        optimizer.zero_grad()
        # loss.backward(retain_graph=True)
        loss.backward()
        optimizer.step()
        functional.reset_net(mlp)

        test_acc = envs[2]['acc']
        if step % 10 == 0:
            pretty_print(
            np.int32(step),
            train_nll.detach().cpu().numpy(),
            train_acc.detach().cpu().numpy(),
            train_penalty.detach().cpu().numpy(),
            test_acc.detach().cpu().numpy()
            )

    final_train_accs.append(train_acc.detach().cpu().numpy())
    final_test_accs.append(test_acc.detach().cpu().numpy())
    print('Final train acc (mean/std across restarts so far):')
    print(np.mean(final_train_accs), np.std(final_train_accs))
    print('Final test acc (mean/std across restarts so far):')
    print(np.mean(final_test_accs), np.std(final_test_accs))

Flags:
	T: 1
	grayscale_model: False
	hidden_dim: 800
	l2_regularizer_weight: 0.00110794568
	lr: 0.0004898536566546834
	n_restarts: 1
	penalty_anneal_iters: 100
	penalty_weight: 5000
	steps: 80
	tau: 2.0
Restart 0
step            train nll       train acc       train penalty   test acc     
0               0.69315         0.50813         0.00000         0.50210      
10              0.69311         0.50813         1.46139e-09     0.50210      
20              0.69308         0.50813         4.29007e-09     0.50210      
30              0.69306         0.50813         6.90887e-09     0.50210      
40              0.69304         0.50813         8.74311e-09     0.50210      
50              0.69303         0.50813         9.89245e-09     0.50210      
60              0.69302         0.50813         1.06884e-08     0.50210      
70              0.69302         0.50813         1.13894e-08     0.50210      
Final train acc (mean/std across restarts so far):
0.508125 0.0
Final test acc (mean

# 高级部分：探索OOD算法

证明、分析算法的因果推理能力和收敛性。